In [1]:
import pandas as pd
import numpy as np

In [2]:
# PATHs

## The directory where the dune data is stored
ORI_DATA_PATH = '../ori_data'

## The directory where the three databases are stored
DATABASE_PATH = '../database'

## The directory where the visualization data for each visualization is stored
VIS_DATA_PATH = '../../data'

## The directory where the scraped tweets data is stored
TWEET_PATH = '../ori_data/tweets'

In [3]:
# read transaction database
tx_db = pd.read_csv('{}/tx_db.csv'.format(DATABASE_PATH), index_col=0)

# read cryptopunk database
punk_db = pd.read_csv('{}/punk_db.csv'.format(DATABASE_PATH), index_col=0)
punk_db['attributes'] = punk_db['attributes'].apply(eval)

# read trader database
trader_db = pd.read_csv('{}/trader_db.csv'.format(DATABASE_PATH), index_col=0)


### 3. Bubble Plot

<img src="../imgs/vis3_example.png" alt="drawing" width="500">

Data format

```
Index
0   tx: a single transaction

Column
1   tx_id
2   date
3   price
4   bubble_size: the attribute count of the punk
5   bubble_color: decided by the skin tone of the punk
6   from: trader_id
7   to: trader_id
8   punk_id
9   punk_img: the image directory of the punk
```

In [4]:
def set_bubble_color(x):
    if x == 'Dark':
        return '#A4031F'
    elif x == 'Medium':
        return '#DB9065'
    elif x == 'Light':
        return '#F2A359'
    else:
        return '#F2DC5D'


def get_dataset_vis3(tx_db, punk_db, trader_db):
    vis3_df = tx_db.copy(deep=True)

    # drop the punk transfer transactions
    vis3_df.dropna(axis=0, inplace=True)

    # set date
    vis3_df['date'] = vis3_df['date_time']  # .apply(lambda x: x.split(' ')[0])

    # set punk_skin_color as bubble color
    vis3_df['punk_skin_color'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'skin_tone'])
    vis3_df['bubble_color'] = vis3_df['punk_skin_color'].apply(
        set_bubble_color)

    # set punk_attr_count as bubble size
    vis3_df['punk_attr_count'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'attr_count'])
    vis3_df.rename(
        columns={'punk_attr_count': 'bubble_size', 'eth_price': 'price'}, inplace=True)

    # set punk features
    vis3_df['img_url'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'img_url'])

    vis3_df['punk_skin_tone'] = vis3_df['punk_id'].apply(
        lambda x: punk_db.loc[x, 'skin_tone'])
    vis3_df['punk_skin_tone'] = vis3_df['punk_skin_tone'].apply(
        lambda x: 'Non-human' if pd.isna(x) else x)

    # set trader addresses
    vis3_df['from_address'] = vis3_df['from'].apply(
        lambda x: trader_db.loc[x, 'address'])
    vis3_df['to_address'] = vis3_df['to'].apply(
        lambda x: trader_db.loc[x, 'address'])
    return vis3_df.loc[:, ['date',  'price', 'bubble_size', 'bubble_color', 'from_address', 'to_address', 'punk_id', 'img_url', 'punk_skin_tone']]


vis3_df = get_dataset_vis3(tx_db, punk_db, trader_db)
vis3_df.to_csv('{}/vis3_data.csv'.format(VIS_DATA_PATH))
vis3_df


,date,price,bubble_size,bubble_color,from_address,to_address,punk_id,img_url,punk_skin_tone
tx_id,,,,,,,,,
0,2017-06-23 21:06:32,0.01,3,#DB9065,\x5b098b00621eda6a96b7a476220661ad265f083f,\xc352b534e8b987e036a93539fd6897f53488e56a,3134,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
1,2017-06-23 21:06:45,0.04,3,#DB9065,\x5b098b00621eda6a96b7a476220661ad265f083f,\x00bd3a6660309fb9e0129b9b777a9ccb9c2869dc,5719,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
2,2017-06-23 21:06:53,0.10,4,#F2DC5D,\xc352b534e8b987e036a93539fd6897f53488e56a,\x00bd9fd57c423a1b1c969823d409156d90974d77,5056,https://www.larvalabs.com/cryptopunks/cryptopu...,Albino
3,2017-06-23 22:06:08,0.06,3,#F2A359,\x5b098b00621eda6a96b7a476220661ad265f083f,\x00bd3a6660309fb9e0129b9b777a9ccb9c2869dc,5624,https://www.larvalabs.com/cryptopunks/cryptopu...,Light
4,2017-06-23 22:06:12,0.03,4,#F2DC5D,\x5b098b00621eda6a96b7a476220661ad265f083f,\x00bd3a6660309fb9e0129b9b777a9ccb9c2869dc,6548,https://www.larvalabs.com/cryptopunks/cryptopu...,Albino
...,...,...,...,...,...,...,...,...,...
16819,2022-05-05 21:05:57,70.00,2,#DB9065,\xe28470253f0c9c7afbae7f61795d6b1ca4644b2f,\x00ebdf16f6f68981e422d09205897a451ad289d6,3314,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
16820,2022-05-06 14:05:57,0.00,3,#DB9065,\xd394cf7e0c9f7c89f81a2870c576c1910974047b,\xed01f8a737813f0bda2d4340d191dbf8c2cbcf30,3095,https://www.larvalabs.com/cryptopunks/cryptopu...,Medium
16821,2022-05-06 17:05:48,58.00,4,#F2A359,\x0256c28028cac3e504f1b6c20bf7b3f4631e0d97,\x1919db36ca2fa2e15f9000fd9cdc2edcf863e685,6995,https://www.larvalabs.com/cryptopunks/cryptopu...,Light


### Vis3.2 3D Scatter Plot

In [25]:
def get_dataset_vis3_2(tx_db, punk_db, trader_db):
    df = punk_db.loc[:, ['type', 'gender', 'skin_tone', 'attr_count', 'img_url', 'avg_price']]
    df = df[df['avg_price'] != 0]
    df['category'] = df.apply(lambda x: x['type']+' '+x['gender'], axis=1)
    df['bubble_color'] = df['skin_tone'].apply(set_bubble_color)
    df['skin_tone'] = df['skin_tone'].apply(lambda x: 'Non-human' if pd.isna(x) else x)
    return df

vis3_2_df = get_dataset_vis3_2(tx_db, punk_db, trader_db)
vis3_2_df.to_csv('{}/vis3_2_data.csv'.format(VIS_DATA_PATH))
vis3_2_df

,type,gender,skin_tone,attr_count,img_url,avg_price,category,bubble_color
punk_id,,,,,,,,
1,Human,Male,Dark,2,https://www.larvalabs.com/cryptopunks/cryptopu...,31.000000,Human Male,#A4031F
14,Human,Female,Dark,3,https://www.larvalabs.com/cryptopunks/cryptopu...,1.750000,Human Female,#A4031F
53,Human,Female,Light,2,https://www.larvalabs.com/cryptopunks/cryptopu...,57.330000,Human Female,#F2A359
89,Human,Female,Dark,1,https://www.larvalabs.com/cryptopunks/cryptopu...,19.990000,Human Female,#A4031F
116,Human,Female,Dark,4,https://www.larvalabs.com/cryptopunks/cryptopu...,1.800000,Human Female,#A4031F
...,...,...,...,...,...,...,...,...
9980,Human,Female,Medium,4,https://www.larvalabs.com/cryptopunks/cryptopu...,33.815000,Human Female,#DB9065
9982,Human,Female,Light,3,https://www.larvalabs.com/cryptopunks/cryptopu...,142.000000,Human Female,#F2A359
9990,Human,Male,Albino,2,https://www.larvalabs.com/cryptopunks/cryptopu...,36.150003,Human Male,#F2DC5D
